In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [27]:
data = pd.read_csv("F:\Data\Experimental/spam.csv", encoding = 'latin-1')

In [28]:
data = data.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)

In [4]:
Y_Train = data.v1
X_Train = data.drop(columns = 'v1')

In [29]:
# Text Preprocessing
import nltk
#nltk.download("all")
from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize

In [30]:
spam_messages = data[data["v1"] == "spam"]["v2"]
ham_messages = data[data["v1"] == "ham"]["v2"]

spam_words = []
ham_words = []

def extractSpamWords(spamMessages):
    global spam_words
    words = [word.lower() for word in word_tokenize(spamMessages) if word.lower() not in stopwords.words("english") and word.lower().isalpha()]
    spam_words = spam_words + words
    
def extractHamWords(hamMessages):
    global ham_words
    words = [word.lower() for word in word_tokenize(hamMessages) if word.lower() not in stopwords.words("english") and word.lower().isalpha()]
    ham_words = ham_words + words

spam_messages.apply(extractSpamWords)
ham_messages.apply(extractHamWords)

0       None
1       None
3       None
4       None
6       None
        ... 
5565    None
5568    None
5569    None
5570    None
5571    None
Name: v2, Length: 4825, dtype: object

In [31]:
data["messageLength"] = data["v2"].apply(len)
data["messageLength"].describe()

count    5572.000000
mean       80.118808
std        59.690841
min         2.000000
25%        36.000000
50%        61.000000
75%       121.000000
max       910.000000
Name: messageLength, dtype: float64

In [32]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")

def cleanText(data):
    
    data = data.translate(str.maketrans('', '', string.punctuation))
    words = [stemmer.stem(word) for word in data.split() if word.lower() not in stopwords.words("english")]
    
    return " ".join(words)

data["v2"] = data["v2"].apply(cleanText)
data.head(n = 10) 

,v1,v2,messageLength
0,ham,go jurong point crazi avail bugi n great world...,111
1,ham,ok lar joke wif u oni,29
2,spam,free entri 2 wkli comp win fa cup final tkts 2...,155
3,ham,u dun say earli hor u c alreadi say,49
4,ham,nah dont think goe usf live around though,61
5,spam,freemsg hey darl 3 week word back id like fun ...,148
6,ham,even brother like speak treat like aid patent,77
7,ham,per request mell mell oru minnaminungint nurun...,160
8,spam,winner valu network custom select receivea å£9...,158
9,spam,mobil 11 month u r entitl updat latest colour ...,154


In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(encoding = "latin-1", strip_accents = "unicode", stop_words = "english")
features = vec.fit_transform(data["v2"])
print(features.shape)

(5572, 7903)


In [34]:
def encodeCategory(cat):
    if cat == "spam":
        return 1
    else:
        return 0
        
data["v1"] = data["v1"].apply(encodeCategory)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, data["v1"], stratify = data["v1"], test_size = 0.2)

In [35]:
X_test

<1115x7903 sparse matrix of type '<class 'numpy.float64'>'
	with 8568 stored elements in Compressed Sparse Row format>

In [37]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import fbeta_score

from sklearn.naive_bayes import MultinomialNB
gaussianNb = MultinomialNB()
gaussianNb.fit(X_train, y_train)

y_pred = gaussianNb.predict(X_test)

print(fbeta_score(y_test, y_pred, beta = 0.5))

0.940099833610649


In [38]:
predict_y = gaussianNb.predict(X_test)
print('Accuracy of NB classifier on test set: {:.2f}'.format(gaussianNb.score(X_test, y_test)))

Accuracy of NB classifier on test set: 0.97


In [43]:
from sklearn.feature_extraction.text import CountVectorizer
comment = ["Check this out"]
vect = vec.transform(comment).toarray()

In [44]:
gaussianNb.predict(vect)

array([0], dtype=int64)